In [10]:
import pandas as pd

In [11]:
df_employee = pd.read_csv("1_data_engineering_employee.csv")
df_employee

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,MonthlyRate,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion
0,41,1,Travel_Rarely,1102,Sales,1,Life Sciences,2,94,3,...,19479,Yes,11,3,1,0,1,6,4,0
1,49,0,Travel_Frequently,279,Research & Development,8,Life Sciences,3,61,2,...,24907,No,23,4,4,1,3,10,7,1
2,37,1,Travel_Rarely,1373,Research & Development,2,Other,4,92,2,...,2396,Yes,15,3,2,0,3,0,0,0
3,33,0,Travel_Frequently,1392,Research & Development,3,Life Sciences,4,56,3,...,23159,Yes,11,3,3,0,3,8,7,3
4,27,0,Travel_Rarely,591,Research & Development,2,Medical,1,40,3,...,16632,No,12,3,4,1,3,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,0,Travel_Frequently,884,Research & Development,23,Medical,3,41,4,...,12290,No,17,3,3,1,3,5,2,0
1466,39,0,Travel_Rarely,613,Research & Development,6,Medical,4,42,2,...,21457,No,15,3,1,1,3,7,7,1
1467,27,0,Travel_Rarely,155,Research & Development,4,Life Sciences,2,87,4,...,5174,Yes,20,4,2,1,3,6,2,0
1468,49,0,Travel_Frequently,1023,Sales,2,Medical,4,63,2,...,13243,No,14,3,4,0,2,9,6,0


In [12]:
%pip install -q scikit-learn
from sklearn.model_selection import train_test_split

Note: you may need to restart the kernel to use updated packages.


In [13]:
df_train, df_test = train_test_split(
    df_employee,
    test_size=0.2,
    stratify=df_employee["Attrition"],
    random_state=42
)

In [14]:
df_employee["Attrition"].value_counts(normalize=True)

Attrition
0    0.838776
1    0.161224
Name: proportion, dtype: float64

In [15]:
df_train["Attrition"].value_counts(normalize=True)

Attrition
0    0.838435
1    0.161565
Name: proportion, dtype: float64

In [16]:
df_test["Attrition"].value_counts(normalize=True)

Attrition
0    0.840136
1    0.159864
Name: proportion, dtype: float64

In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


X_train = df_train.drop("Attrition", axis=1)
y_train = df_train["Attrition"]

X_test = df_test.drop("Attrition", axis=1)
y_test = df_test["Attrition"]

rus = RandomUnderSampler(sampling_strategy=0.8, random_state=42)
X_rus, y_rus = rus.fit_resample(X_train, y_train)

categorical_cols = X_rus.select_dtypes(include="object").columns
numeric_cols = X_rus.select_dtypes(exclude="object").columns

scaler = StandardScaler()
X_num = scaler.fit_transform(X_rus[numeric_cols])

ohe = OneHotEncoder(sparse_output=False)
X_cat = ohe.fit_transform(X_rus[categorical_cols])
X_num = X_rus[numeric_cols].to_numpy()

X_rus_enc = np.hstack([X_num, X_cat])

sm = SMOTE(random_state=42)
X_bal, y_bal = sm.fit_resample(X_rus_enc, y_rus)

ohe_cols = ohe.get_feature_names_out(categorical_cols).tolist()
all_cols = numeric_cols.tolist() + ohe_cols

df_bal = pd.DataFrame(X_bal, columns=all_cols)
df_bal['Attrition'] = y_bal

print(df_bal['Attrition'].value_counts())

df_bal.to_csv("2_training_data_and_feature_engineering_employee.csv", index=False)


Attrition
0    237
1    237
Name: count, dtype: int64
